# **왜 우리 동네에는 스타벅스가 없을까?**

대한민국에서 가장 큰 도시 서울에서 스타벅스가 어떤 전략으로 매장 입지를 선택하는지 살펴보려고 한다. 다음과 같이 스타벅스의 매장 입지에 대한 두 가지 가설을 세우고, 이 가설이 맞는지 데이터 분석을 통해 확인해보고자 한다.

1. 거주 인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
2. 직장인이 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.

## **1. 데이터 수집**
---

스타벅스의 입지전략을 분석하기 위해서는
1. 서울시 내에 출점한 스타벅스들의 위치를 파악해야 한다.
2. 가설을 검증하기 위해 인구통계 데이터 수집도 필요하다.

### **1.1 크롤링을 이용한 서울시 스타벅스 매장 목록 데이터 생성**

주요 데이터는 서울시 내 스타벅스 매장 목록 데이터이다. 전국의 스타벅스 매장 목록은 [스타벅스 코리아](https://www.starbucks.co.kr/store/store_map.do?disp=locale)에 접속해 확인할 수 있다.

**-크롬드라이버를 지우셨다면  [ChromeDriver - WebDriver](https://sites.google.com/a/chromium.org/chromedriver/downloads) 재 다운로드 후 실행-**

In [5]:
pip install selenium # 크롬드라이버는 현재 설치된 크롬의 버전과 동일해야함
Current browser version is 87.0.4280.88 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe

Note: you may need to restart the kernel to use updated packages.


In [1]:
# 라이브러리 임포트
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

+ 크롤링을 실행하기 위해 selenium 라이브러리의 webdriver를 불러왔다.
+ 크롤링으로 가져온 HTML에서 정보를 추출하기 위해 BeautifulSoup 라이브러리를 추가한다.
+ 추출된 데이터를 엑셀로 저장하기 위해 pandas 라이브러리를 추가한다.

In [2]:
# webdriver 실행 후 스타벅스의 지역별 매장 검색 화면에 접속
browser = webdriver.Chrome('C:/Users/bangi/Desktop/chromedriver.exe')# 크롬보다 버전이 높으면 연동을 못시킴(아래와 같이)
# # Message: session not created: This version of ChromeDriver only supports Chrome version 88
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
browser.get(url)

+ webdriver의 Chorme() 함수를 이용해 브라우저를 실행 후 browser 변수에 할당한다.
+ driver.get(url) 명령어를 통해 해당 url로 이동한다.
+ 이 코드를 실행하면 크롬 브라우저가 자동으로 실행된다.

+ 크롬 브라우저에서 '서울' 버튼의 위치를 얻기 위해 개발자 도구를 실행한다.([메뉴] -> [도구 더보기] -> [개발자 도구]).
+ 개발자 도구 상단의 탭에서 [Elements]탭을 클릭하고 왼쪽 끝 사각형에 화살표 표시 버튼을 클릭하면 화면에서 요소를 선택할 수 있다.(밑 사진참고)
![참고](https://user-images.githubusercontent.com/72365693/102745220-dc9b6f80-439e-11eb-8cc1-9c36a00817e0.JPG)
+ '서울'버튼에 마우스를 갖다 대고 클릭하면 개발자 도구 화면에 버튼의 HTML 요소인 < a > 태그가 선택되어 하이라이트된 것을 확인할 수 있다.
+ 하이라이트 표시된 영역에 마우스 커서를 이동한 후 마우스 오른쪽 버튼을 클릭하면 [선택] 메뉴가 나타나고 [Copy] 메뉴에서 [Copy selector]를 클릭하면 전체 HTML 문서 내에서 '서울' 버튼에 해당하는 태그의 구조 정보(CSS selector)가 복사된다.
![z](https://user-images.githubusercontent.com/72365693/102745641-ce9a1e80-439f-11eb-95fd-66bce75ab42f.png)

In [4]:
# webdriver로 '서울' 버튼 요소를 찾아 클릭
# '서울'클릭
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
browser.find_element_by_css_selector(seoul_btn).click()

+ find_element_by_css_selector(seoul_btn)으로 seoul_btn에 저정된 '서울' 버튼의 위치를 선택하고 click()명령으로 해당 버튼을 클릭한다.

In [5]:
# webdriver로 '전체' 버튼 요소를 찾아 클릭
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
browser.find_element_by_css_selector(all_btn).click()

+ 복사한 CSS 셀렉터 값을 all_btn 변수에 저장하고 '전체' 버튼을 클릭하는 코드를 실행한다.
(방법은 위와 동일)

In [6]:
# BeautifulSoup으로 HTML 파서 만들기
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

+ browser.page_source를 통해 크롬 브라우저의 현재 화면에 나타난 웹 페이지의 HTML을 가져올 수 있다.
+ BeautifulSoup(html, 'html.parser')에서 **html.parser**는 HTML 문법을 이해하고 웹 페이지의 정보를 분류하는 역할을 한다.

In [7]:
# select()를 이용해 원하는 HTML 태그를 모두 찾아오기
starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))

531


In [8]:
# 태그 문자열 살펴보기
starbucks_soup_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong>역삼아레나빌딩  <img alt="" class="setStoreFavBtn" data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N" src="//image.istarbucks.co.kr/common/img/store/icon_fav_off.png"/></strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [9]:
# 스타벅스 매장 정보 샘플 확인
starbucks_store = starbucks_soup_list[0]
name = starbucks_store.select('strong')[0].text.strip() # strip 함수를 통해 공백 제거 
lat = starbucks_store['data-lat'].strip()
lng = starbucks_store['data-long'].strip()
store_type = starbucks_store.select('i')[0]['class'][0][4:]
address = str(starbucks_store.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
tel = str(starbucks_store.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

print(name)      # 매장명
print(lat)       # 위도
print(lng)       # 경도
print(store_type)# 매장 타입
print(address)   # 주소
print(tel)       # 전화번호

역삼아레나빌딩
37.501087
127.043069
general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [10]:
# 서울시 스타벅스 매장목록 데이터 만들기
starbucks_list = []
for item in starbucks_soup_list:
    name = item.select('strong')[0].text.strip();
    lat = item['data-lat'].strip()
    lng = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0][4:]
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    
    starbucks_list.append([name, lat, lng, store_type, address, tel])

In [11]:
# pandas의 데이터프레임 생성
columns = ['매장명', '위도', '경도', '매장타입', '주소', '전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list, columns = columns)
seoul_starbucks_df.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [12]:
# 데이터프레임의 요약 정보 확인
seoul_starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531 entries, 0 to 530
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     531 non-null    object
 1   위도      531 non-null    object
 2   경도      531 non-null    object
 3   매장타입    531 non-null    object
 4   주소      531 non-null    object
 5   전화번호    531 non-null    object
dtypes: object(6)
memory usage: 25.0+ KB


In [21]:
%pwd # 경로가 맞지 않으면 FileCreateError: [Errno 2] No such file or directory: 'example/seoul_starbucks_list.xlsx' 같은 에러 발생

'C:\\Users\\bangi\\Desktop\\python_portfolio\\크롤링 데이터베이스'

In [13]:
# 엑셀로 저장
seoul_starbucks_df.to_excel('example/seoul_starbucks_list.xlsx', index = False)

### **1.2 서울열린데이터광장의 OPEN API를 활용한 공공데이터 수집**

[서울열린데이터광장](http://data.seoul.go.kr/)사이트에서 데이터를 내려받거나 OPEN API를 통해 획득할 수 있다.

#### **서울열린데이터광장 인증키 발급**

[인증키 신청](http://data.seoul.go.kr/together/mypage/actkeyMain.do) -> 인증키 복사 후 -> 주소창에 http://openapi.seoul.go.kr:8088/{인증키}/json/SdeTlSccoSigW/1/25 작성 후 검색

**변수명**|**타입**|**변수설명**|**값설명**
:---:|:---:|:---:|:---:
KEY | String(필수) | 인증키 | 서울열린데이터광장에서 발급된 인증키
TYPE | String(필수) | 요청파일타입 | xml/xmlf/xls/json 타입 지원
SERVICE | String(필수) | 서비스명 | SdeTlSccoSigW
START_INDEX | String(필수) | 요청 시작 위치 | 숫자 입력
END_INDEX | String(필수) | 요청 종료 위치 | 숫자 입력
OBJECTID | String(선택) | 순번 | 정수 입력

**OBJECTID**는 필수가 아니기 때문에 별도로 값을 입력하지 않아도 된다.

In [14]:
# 라이브러리 임포트
import requests
import pandas as pd

In [ ]:
# API 호출을 위한 URL 변수 선언
# 시군구 목록을 가져오는 API 호출 URL

url = 'http://openapi.seoul.go.kr:8088/********************************/json/SdeTlSccoSigW/1/25'
print(url)

In [16]:
# 서울열린데이터광장 OPEN API 호출
result_dict = requests.get(url).json()
result_dict

{'SdeTlSccoSigW': {'list_total_count': 25,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'OBJECTID': 1.0,
    'SIG_CD': '11320',
    'SIG_KOR_NM': '도봉구',
    'SIG_ENG_NM': 'Dobong-gu',
    'ESRI_PK': 0.0,
    'LAT': '37.6658609',
    'LNG': '127.0317674'},
   {'OBJECTID': 2.0,
    'SIG_CD': '11380',
    'SIG_KOR_NM': '은평구',
    'SIG_ENG_NM': 'Eunpyeong-gu',
    'ESRI_PK': 1.0,
    'LAT': '37.6176125',
    'LNG': '126.9227004'},
   {'OBJECTID': 3.0,
    'SIG_CD': '11230',
    'SIG_KOR_NM': '동대문구',
    'SIG_ENG_NM': 'Dongdaemun-gu',
    'ESRI_PK': 2.0,
    'LAT': '37.5838012',
    'LNG': '127.0507003'},
   {'OBJECTID': 4.0,
    'SIG_CD': '11590',
    'SIG_KOR_NM': '동작구',
    'SIG_ENG_NM': 'Dongjak-gu',
    'ESRI_PK': 3.0,
    'LAT': '37.4965037',
    'LNG': '126.9443073'},
   {'OBJECTID': 5.0,
    'SIG_CD': '11545',
    'SIG_KOR_NM': '금천구',
    'SIG_ENG_NM': 'Geumcheon-gu',
    'ESRI_PK': 4.0,
    'LAT': '37.4600969',
    'LNG': '126.9001546'},
   {'OBJECTID': 6.0

In [17]:
# 시군구 목록 결과 딕셔너리 데이터의 값 가져오기
result_data = result_dict['SdeTlSccoSigW']

print(result_data['list_total_count'])
print(result_data['RESULT'])
print(result_data['row'][0])

25
{'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}
{'OBJECTID': 1.0, 'SIG_CD': '11320', 'SIG_KOR_NM': '도봉구', 'SIG_ENG_NM': 'Dobong-gu', 'ESRI_PK': 0.0, 'LAT': '37.6658609', 'LNG': '127.0317674'}


In [18]:
# 결과 데이터를 pandas의 데이터프레임으로 변환
data_list = result_data['row']
sample_df = pd.DataFrame(data_list)
sample_df.head()

,OBJECTID,SIG_CD,SIG_KOR_NM,SIG_ENG_NM,ESRI_PK,LAT,LNG
0,1.0,11320,도봉구,Dobong-gu,0.0,37.6658609,127.0317674
1,2.0,11380,은평구,Eunpyeong-gu,1.0,37.6176125,126.9227004
2,3.0,11230,동대문구,Dongdaemun-gu,2.0,37.5838012,127.0507003
3,4.0,11590,동작구,Dongjak-gu,3.0,37.4965037,126.9443073
4,5.0,11545,금천구,Geumcheon-gu,4.0,37.4600969,126.9001546


#### **서울열린데이터광장 OPEN API를 호출하는 함수 작성**

In [19]:
# 서울열린데이터광장 OPEN API 호출을 위한 공통 함수
def seoul_open_api_data(url, service):
    data_list = None
    try:
        result_dict = requests.get(url).json()
        result_data = result_dict[service]
        code = result_data['RESULT']['CODE']
        if code == 'INFO-000':
            data_list = result_data['row']
    except:
        pass
    return data_list

#### **시군구 목록 데이터를 수집한 후 엑셀 다운로드하기**

In [20]:
# 시군구 목록 데이터를 수집한 후 엑셀 다운로드하기
sgg_url = 'http://openapi.seoul.go.kr:8088/******************************************/json/SdeTlSccoSigW/1/25'
sgg_data_list = seoul_open_api_data(url, 'SdeTlSccoSigW')
sgg_data_list[0]

{'OBJECTID': 1.0,
 'SIG_CD': '11320',
 'SIG_KOR_NM': '도봉구',
 'SIG_ENG_NM': 'Dobong-gu',
 'ESRI_PK': 0.0,
 'LAT': '37.6658609',
 'LNG': '127.0317674'}

In [21]:
# 결과 데이터를 pandas의 데이터프레임으로 변환
columns = ['SIG_CD','SIG_KOR_NM','LAT','LNG']
sgg_df = pd.DataFrame(data=sgg_data_list, columns=columns)
sgg_df.head()

,SIG_CD,SIG_KOR_NM,LAT,LNG
0,11320,도봉구,37.6658609,127.0317674
1,11380,은평구,37.6176125,126.9227004
2,11230,동대문구,37.5838012,127.0507003
3,11590,동작구,37.4965037,126.9443073
4,11545,금천구,37.4600969,126.9001546


In [22]:
# 데이터프레임의 컬럼명 변경
import warnings
warnings.filterwarnings('ignore')

sgg_df.columns = ['시군구코드','시군구명','위도','경도']
sgg_df.head()

,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.6658609,127.0317674
1,11380,은평구,37.6176125,126.9227004
2,11230,동대문구,37.5838012,127.0507003
3,11590,동작구,37.4965037,126.9443073
4,11545,금천구,37.4600969,126.9001546


In [23]:
# 데이터프레임의 요약 정보 확인
sgg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구코드   25 non-null     object
 1   시군구명    25 non-null     object
 2   위도      25 non-null     object
 3   경도      25 non-null     object
dtypes: object(4)
memory usage: 928.0+ bytes


In [24]:
# 엑셀로 저장
sgg_df.to_excel('example/seoul_sgg_list.xlsx', index = False)

In [25]:
# 서울시 시군구별 인구수 데이터 가져오기
sgg_pop = pd.read_excel('example/seoul_sgg_list.xlsx')
sgg_pop.head()

,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.665861,127.031767
1,11380,은평구,37.617612,126.922700
2,11230,동대문구,37.583801,127.050700
3,11590,동작구,37.496504,126.944307
4,11545,금천구,37.460097,126.900155


## **2. 데이터 전처리**
---

지금까지 수집한 데이터를 이용하여 시군구를 기준으로 분석해 보겠다.

두 개 이상의 데이터를 병합하기 위해서는 공통된 기준값이 필요하며, 이를 **키(Key)**라고 한다.

### **2.1 서울시 스타벅스 매장 목록, 시군구명, 시군구코드 추가**

In [26]:
# 서울시 스타벅스 매장 목록이 담긴 엑셀 파일 불러오기
seoul_starbucks = pd.read_excel('example/seoul_starbucks_list.xlsx', header = 0) 
seoul_starbucks.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


스타벅스의 주소 정보에 '서울특별시 xx구'라는 시군구명 정보가 포함돼 있으니 주소에서 시군구명 정보를 추출하겠다.

In [27]:
# 스타벅스 주소 정보에서 시군구명 추출
sgg_names = []
for address in seoul_starbucks['주소']:
    sgg = address.split()[1]
    sgg_names.append(sgg)
seoul_starbucks['시군구명'] = sgg_names
seoul_starbucks.head()

,매장명,위도,경도,매장타입,주소,전화번호,시군구명
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232,강남구
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,강남구
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,강남구


시군구명과 시군구 코드가 추가된 서울시 스타벅스 목록 데이터는 데이터 시각화와 분석에 활용할 최종 데이터이다. 이 데이터를 다른 엑셀 파일로 저장해 보자.

In [28]:
# 엑셀로 저장
seoul_starbucks.to_excel('example/2_seoul_starbucks_list.xlsx', index = False)

### **2.2 스타벅스 분석 데이터 만들기**

In [29]:
# 라이브러리 임포트
import pandas as pd

In [30]:
# 시군구 목록 데이터 불러오기
seoul_sgg = pd.read_excel('example/seoul_sgg_list.xlsx')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.665861,127.031767
1,11380,은평구,37.617612,126.922700
2,11230,동대문구,37.583801,127.050700
3,11590,동작구,37.496504,126.944307
4,11545,금천구,37.460097,126.900155


In [31]:
# 서울시 스타벅스 매장 목록 데이터 불러오기
seoul_starbucks = pd.read_excel('example/2_seoul_starbucks_list.xlsx')
seoul_starbucks.head()

,매장명,위도,경도,매장타입,주소,전화번호,시군구명
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232,강남구
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,강남구
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,강남구


In [32]:
# 시군구별 스타벅스 매장 수 세기
starbucks_sgg_count = seoul_starbucks.pivot_table(
    index = '시군구명',
    values = '매장명',
    aggfunc = 'count').rename(columns = {'매장명':'스타벅스_매장수'})
starbucks_sgg_count.head()

,스타벅스_매장수
시군구명,
강남구,81
강동구,15
강북구,5
강서구,16
관악구,11


In [33]:
# 서울시 시군구 목록 데이터에 스타벅스 매장 수 데이터를 병합
seoul_sgg = pd.merge(seoul_sgg, starbucks_sgg_count, how = 'left', on = '시군구명')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장수
0,11320,도봉구,37.665861,127.031767,2
1,11380,은평구,37.617612,126.922700,8
2,11230,동대문구,37.583801,127.050700,8
3,11590,동작구,37.496504,126.944307,11
4,11545,금천구,37.460097,126.900155,11


## **3. 데이터 시각화**
---

앞에서 정리한 데이터를 시각화 해보겠다. 먼저, 스타벅스 매장 위치를 지도에 시각화해 서울시 내에서 스타벅스 매장이 어떻게 분포돼 있는지, 일반 매장과 리저브, 드라이브스루 매장 분포에 어떤 차이가 있는지 확인해 보겠다.

### **3.1 스타벅스 매장분포 시각화**

시각화에 필요한 라이브러리를 임포트 한다. 그 전에 folium을 설치 한다.

In [50]:
# folium설치
! pip install folium

In [52]:
# idx설치
! pip install idx

  Created wheel for idx: filename=idx-0.0.0-py3-none-any.whl size=1347 sha256=0abfc89277ea63ba37ac639d89589739f5b1e56405e14d99561eced7bd006c87
  Stored in directory: c:\users\bangi\appdata\local\pip\cache\wheels\27\ad\59\5f540b3e1893b63e6f980a710c806cf438253702efd394ee82
Successfully built idx


In [34]:
# 라이브러리 임포트
import pandas as pd  # 수집한 데이터 불러오고 전처리 하기 위해 임포트
import folium        # 시각화하기 위해 임포트
import json          # GeoJson 타입의 지도파일을 사용하기 위해 임포트
import idx           # index 호출

In [35]:
# 서울시 스타벅스 매장 목록 데이터 불러오기
seoul_starbucks = pd.read_excel('example/2_seoul_starbucks_list.xlsx')
seoul_starbucks.head()

,매장명,위도,경도,매장타입,주소,전화번호,시군구명
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232,강남구
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,강남구
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,강남구


In [36]:
# folium을 이용한 지도 생성
starbucks_map = folium.Map(
    location = [37.501087, 127.043069],
    tiles = 'Stamen Terrain',
    zoom_start = 11)
starbucks_map

In [37]:
# 지도에 스타벅스 매장 위치를 나타내는 서클 마커 그리기
for idx in seoul_starbucks.index:
    lat = seoul_starbucks.loc[idx, '위도']
    lng = seoul_starbucks.loc[idx, '경도']

    folium.CircleMarker(
        location=[lat, lng],
        fill = True, 
        fill_color='green', 
        fill_opacity=1,
        color='yellow', 
        weight=1,
        radius=3
    ).add_to(starbucks_map)

starbucks_map

In [38]:
# 스타벅스 매장 타입별 위치 서클 마커 그리기
starbucks_map2 = folium.Map(
    location = [37.501087, 127.043069],
    tiles = 'Stamen Terrain',
    zoom_start = 11)

for idx in seoul_starbucks.index:
    lat = seoul_starbucks.loc[idx, '위도']
    lng = seoul_starbucks.loc[idx, '경도']
    store_type = seoul_starbucks.loc[idx, '매장타입']
    
    # 매장 타입별 색상 선택을 위한 조건문
    fillColor = ''
    if store_type == 'general':
        fillColor = 'gray'
        size = 3
    elif store_type == 'reserve':
        fillColor = 'blue'
        size = 4
    elif store_type == 'generalDT':
        fillColor = 'green'
        size = 5
        
    folium.CircleMarker(
        location = [lat,lng],
        color = fillColor,
        fill = True,
        fill_color = fillColor,
        fill_opacity = 1,
        weight = 1,
        radius = size).add_to(starbucks_map2)
    
starbucks_map2